In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from glob import glob

# --- CONFIGURATION --- #
DATA_FOLDER = r"C:\Users\bmoha\Desktop\DataCollection\2150\heat_logs\2025-6-5_13161"
OUTPUT_FOLDER = os.path.join(DATA_FOLDER, "output")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- DEBUG: Show all file types --- #
all_files = glob(os.path.join(DATA_FOLDER, "*.*"))
print(f"\n🔍 All files in folder: {len(all_files)} total")
for f in all_files[:5]:
    print(" -", os.path.basename(f))

# --- Load only .xlsx files larger than 5KB --- #
raw_excel_files = glob(os.path.join(DATA_FOLDER, "*.xlsx"))
print(f"\n📄 .xlsx files found: {len(raw_excel_files)}")

excel_files = [f for f in raw_excel_files if os.path.getsize(f) > 5 * 1024]
print(f"✅ Excel files > 5 KB: {len(excel_files)}")

if not excel_files:
    print("⚠️ No valid Excel files found. Check path, file extensions, or size filter.")
else:
    print("\n📂 Starting processing...")

# --- Loop through files --- #
for file_path in excel_files:
    file_name = os.path.basename(file_path)
    print(f"\n🔧 Processing: {file_name}")

    try:
        df = pd.read_excel(file_path)

        # Required columns check
        required_cols = {
            'time', 'position_x', 'position_y', 'position_z',
            'rotation_yaw', 'rotation_pitch', 'rotation_roll', 'Aname'
        }

        if not required_cols.issubset(df.columns):
            print(f"❌ Skipped {file_name}: Missing required columns.")
            continue

        # Filter head data
        head_df = df[df['Aname'].str.endswith('Head', na=False)]
        chris_head = head_df[head_df['Aname'].str.contains('BP_Chris_28', na=False)]
        pawn_head = head_df[head_df['Aname'].str.contains('myMRPawn', na=False)]

        if chris_head.empty or pawn_head.empty:
            print(f"⚠️ Skipped {file_name}: No head data found for one or both characters.")
            continue

        # Merge on time
        merged = pd.merge(chris_head, pawn_head, on='time', suffixes=('_chris', '_pawn'))

        if merged.empty:
            print(f"⚠️ Skipped {file_name}: No common time entries to merge.")
            continue

        # Compute distance
        merged['head_distance'] = np.sqrt(
            (merged['position_x_chris'] - merged['position_x_pawn'])**2 +
            (merged['position_y_chris'] - merged['position_y_pawn'])**2 +
            (merged['position_z_chris'] - merged['position_z_pawn'])**2
        )

        # Plot distance
        plt.figure(figsize=(10, 4))
        plt.plot(merged['time'], merged['head_distance'], label='Head Distance', color='green')
        plt.title(f'Head Distance Over Time\n{file_name}', fontsize=10)
        plt.xlabel('Time')
        plt.ylabel('Distance (units)')
        plt.grid(True)
        plt.tight_layout()

        # Save plot
        output_path = os.path.join(OUTPUT_FOLDER, file_name.replace('.xlsx', '_head_distance.png'))
        plt.savefig(output_path)
        plt.close()

        print(f"✅ Plot saved to: {output_path}")

    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")


🔍 All files in folder: 0 total

📄 .xlsx files found: 0
✅ Excel files > 5 KB: 0
⚠️ No valid Excel files found. Check path, file extensions, or size filter.
